In [1]:
import dspy
from dslmodel import DSLModel, init_instant
from pydantic import Field

In [2]:
paper_abstract = """DSPy: Compiling Declarative Language Model Calls into Self-Improving Pipelines
Omar Khattab, Arnav Singhvi, Paridhi Maheshwari, Zhiyuan Zhang, Keshav Santhanam, Sri Vardhamanan, Saiful Haq, Ashutosh Sharma, Thomas T. Joshi, Hanna Moazam, Heather Miller, Matei Zaharia, Christopher Potts
The ML community is rapidly exploring techniques for prompting language models (LMs) and for stacking them into pipelines that solve complex tasks. Unfortunately, existing LM pipelines are typically implemented using hard-coded "prompt templates", i.e. lengthy strings discovered via trial and error. Toward a more systematic approach for developing and optimizing LM pipelines, we introduce DSPy, a programming model that abstracts LM pipelines as text transformation graphs, i.e. imperative computational graphs where LMs are invoked through declarative modules. DSPy modules are parameterized, meaning they can learn (by creating and collecting demonstrations) how to apply compositions of prompting, finetuning, augmentation, and reasoning techniques. We design a compiler that will optimize any DSPy pipeline to maximize a given metric. We conduct two case studies, showing that succinct DSPy programs can express and optimize sophisticated LM pipelines that reason about math word problems, tackle multi-hop retrieval, answer complex questions, and control agent loops. Within minutes of compiling, a few lines of DSPy allow GPT-3.5 and llama2-13b-chat to self-bootstrap pipelines that outperform standard few-shot prompting (generally by over 25% and 65%, respectively) and pipelines with expert-created demonstrations (by up to 5-46% and 16-40%, respectively). On top of that, DSPy programs compiled to open and relatively small LMs like 770M-parameter T5 and llama2-13b-chat are competitive with approaches that rely on expert-written prompt chains for proprietary GPT-3.5. DSPy is available at this https URL
"""

class ArxivPaper(DSLModel):
  lead_author: str = Field(description="The lead author of the paper")
  co_authors: list[str] = Field(description="The co-authors of the paper")
  abstract_summary: str = Field(description="The summary of the paper abstract")

init_instant()
response = ArxivPaper.from_prompt(paper_abstract)
print(response.lead_author)
print(response.co_authors)
print(response.abstract_summary)

Omar Khattab
['Arnav Singhvi', 'Paridhi Maheshwari', 'Zhiyuan Zhang', 'Keshav Santhanam', 'Sri Vardhamanan', 'Saiful Haq', 'Ashutosh Sharma', 'Thomas T. Joshi', 'Hanna Moazam', 'Heather Miller', 'Matei Zaharia', 'Christopher Potts']
The paper introduces DSPy, a programming model that abstracts language model pipelines as text transformation graphs, and designs a compiler to optimize these pipelines.


In [7]:
class SentimentModel(DSLModel):
    sentiment: str = Field(description="The sentiment of the sentence. LOWER CASE.", max_length=5)
    confidence: float

# Example sentence for sentiment classification
sentence = "This is a bad experience!"

# Define the DSPy signature as a string
signature = "sentence -> sentiment"

init_instant()

# Since no predictor_class is provided, dspy.Predict will be used by default
response = SentimentModel.from_signature(signature=signature, sentence=sentence)

# Access the sentiment from the model instance
print(response)

2024-10-08T03:17:45.398276Z [error    ] AssertionError: You need to create a kwargs dict for SentimentModel
Validation error:
1 validation error for SentimentModel
sentiment
  String should have at most 5 characters [type=string_too_long, input_value='Negative', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/string_too_long [dspy.primitives.assertions] filename=assertions.py lineno=88
Error during validation: You need to create a kwargs dict for SentimentModel
Validation error:
1 validation error for SentimentModel
sentiment
  String should have at most 5 characters [type=string_too_long, input_value='Negative', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/string_too_long
Output:
{'sentiment': 'Negative', 'confidence': 0.5}


sentiment='neg' confidence=0.5


In [8]:
# Define a DSPy signature class for summarization
class SummarizationSignature(dspy.Signature):
    """Summarize the document."""
    document = dspy.InputField()
    summary = dspy.OutputField(desc="10 words or less")

class SummarizationModel(DSLModel):
    summary: str = Field(description="The summary of the document.")

# Example document for summarization
document = """The first of the month is coming, we have to get money, we have no choice. It cost money to eat and they don’t want you to eat. Life is what you make it, so let’s make it. They don’t want us to win. Look at the sunset, life is amazing, life is beautiful, life is what you make it. Don’t ever play yourself. Hammock talk come soon. Special cloth alert. Learning is cool, but knowing is better, and I know the key to success. Eliptical talk. Let’s see what Chef Dee got that they don’t want us to eat. Give thanks to the most high. Give thanks to the most high.

The key is to drink coconut, fresh coconut, trust me. Congratulations, you played yourself. The weather is amazing, walk with me through the pathway of more success. Take this journey with me, Lion! Egg whites, turkey sausage, wheat toast, water. Of course they don’t want us to eat our breakfast, so we are going to enjoy our breakfast. The key to success is to keep your head above the water, never give up. The key is to enjoy life, because they don’t want you to enjoy life. I promise you, they don’t want you to jetski, they don’t want you to smile.

Look at the sunset, life is amazing, life is beautiful, life is what you make it. They don’t want us to win. They key is to have every key, the key to open every door. Another one. Always remember in the jungle there’s a lot of they in there, after you overcome they, you will make it to paradise. Don’t ever play yourself. They never said winning was easy. Some people can’t handle success, I can. To be successful you’ve got to work hard, to make history, simple, you’ve got to make it. They key is to have every key, the key to open every door."""

# Using a signature class and providing a custom predictor class (e.g., dspy.ChainOfThought)
response = SummarizationModel.from_signature(signature=SummarizationSignature, 
                                             predictor_class=dspy.ChainOfThought, 
                                             document=document,
                                             verbose=True)

# Access the summary from the model instance
print(response)

Prompt: Prediction(
    reasoning="The document appears to be a motivational speech or poem, encouraging the reader to take control of their life, be successful, and not let others bring them down. It emphasizes the importance of perseverance, hard work, and being mindful of one's surroundings.",
    summary='Motivational speech on success and perseverance.'
)
Prompt: Prediction(
    reasoning="The document appears to be a motivational speech or poem, encouraging the reader to take control of their life, be successful, and not let others bring them down. It emphasizes the importance of perseverance, hard work, and being mindful of one's surroundings.",
    summary='Motivational speech on success and perseverance.'
)
summary='Motivational speech on success and perseverance.'
